In [ ]:
import torch

cuda_available = torch.cuda.is_available()

print(f"Torch version: {torch.__version__}\nCuda is available: {cuda_available}")
assert cuda_available

Torch version: 2.2.1+cu121
Cuda is available: True


In [ ]:


!pip install rdflib pymongo openai > /dev/null

In [ ]:
!pip install bitsandbytes
!pip install accelerate
!pip install trl torch peft datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 744.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

# Get the datasets

In [ ]:
!wget https://raw.githubusercontent.com/KGQA/QALD_9_plus/main/data/qald_9_plus_test_wikidata.json > /dev/null # qald-9-plus-test
!wget https://raw.githubusercontent.com/KGQA/QALD_9_plus/main/data/qald_9_plus_train_wikidata.json > /dev/null # qald-9-plus train
!wget https://raw.githubusercontent.com/KGQA/QALD-10/main/data/qald_10/qald_10.json > /dev/null # qald-10
!wget https://raw.githubusercontent.com/vladislavneon/RuBQ/master/RuBQ_2.0/RuBQ_2.0_dev.json > /dev/null # rubq 2.0 dev
!wget https://raw.githubusercontent.com/vladislavneon/RuBQ/master/RuBQ_2.0/RuBQ_2.0_test.json > /dev/null # rubq 2.0 test

--2024-03-17 13:25:51--  https://raw.githubusercontent.com/KGQA/QALD_9_plus/main/data/qald_9_plus_test_wikidata.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8912790 (8.5M) [text/plain]
Saving to: ‘qald_9_plus_test_wikidata.json’

qald_9_plus_test_wi 100%[===================>]   8.50M  --.-KB/s    in 0.05s   

2024-03-17 13:25:52 (173 MB/s) - ‘qald_9_plus_test_wikidata.json’ saved [8912790/8912790]

--2024-03-17 13:25:52--  https://raw.githubusercontent.com/KGQA/QALD_9_plus/main/data/qald_9_plus_train_wikidata.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request se

# Import dependencies

In [ ]:
import re
import os
import json
import logging
import requests
import pandas as pd
from time import sleep
from rdflib import URIRef
from datetime import datetime
from pymongo import MongoClient
from pyparsing import ParseResults
from rdflib.plugins.sparql.parser import parseQuery

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

#Model Quantization and BitsAndBytes - https://medium.com/@rakeshrajpurohit/model-quantization-with-hugging-face-transformers-and-bitsandbytes-integration-b4c9983e8996

#Quantization in 4-bit
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# Read datasets

In [ ]:
files = [f for f in os.listdir() if ".json" in f]

data_dict = {}

for file in files:
  with open(file) as f:
    data = json.load(f)
  data_dict[file] = data

# Setup DB and Model


In [ ]:
mongo_client = MongoClient('141.57.8.18:40200',
    username='admin',
    password='admin123',
    authSource='admin'
)

db = mongo_client['QAfiltering']

MODEL = "perevalov/Mistral-7B-instruct-v0.2-SPARQL2NL" # "mistralai/Mistral-7B-Instruct-v0.2"

model_bin = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map='auto',
    #load_in_8bit=True, #Quantization in 8-bit
    quantization_config=nf4_config, #Quantization in 4-bit
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [ ]:
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [ ]:
REPLACE = (('[', ''), (']', ''), (' ', '_'), (',', ''), ("'", ''))

WIKIDATA_PREFIXES = ['wd:', 'wdt:', 'p:', 'ps:', 'pq:',]

FIXED_LABELS = {
    'rdfs:label': 'label',
    'http://www.w3.org/2000/01/rdf-schema#label': 'label',
    'skos:altLabel': 'alternative label',
    'xsd:integer': 'integer',
    'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': 'type',
    'http://www.w3.org/2001/XMLSchema#integer': 'integer',
    'http://www.w3.org/2001/XMLSchema#gYear': 'year',
    'http://www.w3.org/2001/XMLSchema#double': 'double',
}

headers = {
    'User-Agent': 'wiki_parser_online/0.17.1 (https://deeppavlov.ai; info@deeppavlov.ai) deeppavlov/0.17.1',
}

WIKIDATA = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'

In [ ]:
from operator import truediv
def extract_entities_recursive(parsed):
    results = []

    if isinstance(parsed, URIRef):
        results += [str(parsed)]

    if isinstance(parsed, (list, ParseResults)):
        for i in list(parsed):
            results += extract_entities_recursive(i)

    if isinstance(parsed, dict):
        if 'prefix' in parsed and 'localname' in parsed:
            results += [f'{parsed["prefix"]}:{parsed["localname"]}']
        else:
            for i in parsed.values():
                results += extract_entities_recursive(i)

    return results

def extract_entities(sparql):
    try:
        return extract_entities_recursive(parseQuery(sparql)[1]['where'])
    except:
        return []

def execute(query: str, endpoint_url: str = WIKIDATA):
    """
    Send query direct to wikidata.

    query: SPARQL query
    endpoint_url: endpoint of wikidata query service

    execute(): json response
    """
    try:
        r = requests.get(endpoint_url, headers=headers, params = {'format': 'json', 'query': query})

        if r.status_code == 200:
            return r.json()

    except Exception as e:
        logger.error(f'Exception in function "execute": {e}')

    return None

def query_wikidata(query: str, repeat: int = 3, timeout: float = 10.0):
    """
    Send query direct to wikidata.
    query: SPARQL query
    query_wikidata(): list of dicts
    """
    while repeat > 0:
        try:
            return execute(query)['results']['bindings']

        except Exception as e:
            logger.error(f'Exception in function "query_wikidata": {e}')
            sleep(timeout)
            repeat -= 1

    return None

def query_wikidata_label(uri: str, lang: str='en') -> str:
    query = (
        'SELECT ?label WHERE {',
        '  {',
        f'    {uri} rdfs:label ?label .',
        '  }',
        '  UNION',
        '  {',
        f'    {uri} owl:sameAs+ ?redirect .',
        '    ?redirect rdfs:label ?label .',
        '  }',
        f'  FILTER (lang(?label) = "{lang}")',
        '} LIMIT 1'
    )

    try:
        data = query_wikidata('\n'.join(query))
        return data[0]['label']['value']
    except KeyError:
        raise
    except Exception as e:
        # print(f'Exception in function "query_wikidata_label": {e}')
        # print(f'Input parameters: {uri}, {lang}')
        return None

def get_wikidata_label_cached(cache, uri: str, lang: str='en') -> str:
    label = cache.wikidata_labels.find_one({'uri': uri, 'lang': lang})

    if label:
        logger.debug(f"Found label in cache {label}")
        return label['label']

    prefix = 'wd' # it works despite property or entity

    try:
        label = query_wikidata_label(f'{prefix}:{uri}', lang)
        logger.debug(f"get_wikidata_label_cached label: {str(label)}, lang: {str(lang)}")
        if not label:
            label = None

        cache.wikidata_labels.insert_one({ 'uri': uri, 'lang': lang, 'label': label })

    except KeyboardInterrupt:
        raise
    except Exception as e:
        # print(f'Exception in function "get_wikidata_label_cached": {e}')
        # print(f'Input parameters: {uri}, {lang}')
        return None
    logger.debug(f"returning label {label} from  get_wikidata_label_cached")

    return label

def get_wikidata_label(cache, literal, lang='en'):
    literal = literal.strip('(<>).')

    if literal in FIXED_LABELS:
        return FIXED_LABELS[literal]

    if not any(i in literal for i in WIKIDATA_PREFIXES+['http']):
        if "xsd:" not in literal:
            return literal
        else:
            return None

    if any(literal.startswith(i) for i in WIKIDATA_PREFIXES):
        w_id = literal.split(':')[-1]
    else:
        parts = literal.split('/')
        w_id = parts[-1][:-1] if parts[-1].endswith(">") else parts[-1]

    label = get_wikidata_label_cached(cache, w_id, lang)

    return label

def get_question_by_language(item, dataset='qald', lang='en'):
  q_list = []
  if dataset == 'qald':
    for q in item['question']:
      if q['language'] == lang:
        q_list.append(q['string'])
    return q_list
  elif dataset == 'rubq':
    if lang == 'en':
      q_list.append(item["question_eng"])
    elif lang == 'ru':
      q_list.append(item["question_text"])
    else:
      assert False
    return q_list
  else:
    assert False

def make_the_prompt(cache, query, prompt_template, lang='en', dataset=None):
    import random
    entities = extract_entities(query)

    list_labels = []
    for i in entities:
        label = get_wikidata_label(cache, i, lang)
        if label:
            list_labels.append(prompt_template[lang]['list'].format(uri=i, uriLabel=label))

    if prompt_template["type"] == "ZERO_SHOT":
      prompt = prompt_template[lang]["head"].format(query=query) + "\n ".join(l for l in list_labels) + prompt_template[lang]["tail"]
    elif prompt_template["type"] == "ONE_SHOT":
      queries = [q["query"]["sparql"] for q in dataset].copy()
      questions = [get_question_by_language(q, 'qald', lang) for q in dataset].copy()

      r_idx = queries.index(query)
      queries.pop(r_idx) # remove the main query
      questions.pop(r_idx) # remove the main query

      c_idx = random.choice([i for i in range(len(queries))]) # choice a random query to make a one shot
      shot_query = queries[c_idx]
      gold_standard = questions[c_idx]

      shot = make_the_prompt(cache, shot_query, {"type": "ZERO_SHOT", lang: prompt_template[lang]}, lang=lang)
      shot += ". " + random.choice(gold_standard)
      prompt = prompt_template[lang]["shot"].format(shot=shot) + prompt_template[lang]["head"].format(query=query) + "\n ".join(l for l in list_labels) + prompt_template[lang]["tail"]

    return prompt

def find_in_cache(collection_name: str, filter_dict: dict):
    try:
        result = db[collection_name].find_one(filter_dict)
        if result:
            return result
        else:
            return None
    except Exception as e:
        print(str(e))
        return None

def cache_gpt(model: str, prompt: dict, result: dict):
    try:
        document = {
            'prompt': prompt,
            'result': result,
            'date': datetime.now()
        }
        db[model].insert_one(document)
    except Exception as e:
        print(str(e))

def ask_llm(prompt, model=MODEL):
  cached_result = find_in_cache(model, {"prompt": prompt})
  if cached_result:
    return cached_result["result"].replace("</s>", "").strip()

  messages = [
      {"role": "user", "content": prompt}
  ]

  model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")

  generated_ids = model_bin.generate(model_inputs, max_new_tokens=1000, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  result = decoded[0].split("[/INST]")[-1]

  cache_gpt(model, prompt, result)

  return result.replace("</s>", "").strip()

In [ ]:
ZERO_SHOT_PROMPT = {
    'type': "ZERO_SHOT",
    'en': {
        'head': 'Having a SPARQL query: {query} \n Where:\n ',
        'list': '{uri} has human-readable name "{uriLabel}."',
        'tail': '\n Transform the SPARQL query to a natural language question. Output just the transformed question'
    },
    'ru': {
        'head': 'Имея следующий SPARQL запрос: {query} \n Где:\n ',
        'list': '{uri} именуется как "{uriLabel}."',
        'tail': '\n Трансформируй SPARQL запрос в вопрос на естественном языке. Выведи только транфсормируемый вопрос'
    },
    'de': {
        'head': 'Gegeben ist die SPARQL-Anfrage: {query} \n Dabei gilt:\n ',
        'list': 'Die Bezeichnung von {uri} ist "{uriLabel}."',
        'tail': '\n Transformiere die SPARQL-Anfrage in eine Frage in natürlicher Sprache. Gib nur die transformierte Frage aus.'
    }
}

ONE_SHOT_PROMPT = {
    'type': "ONE_SHOT",
    'en': {
        'shot': "---- Start Example ---- \n {shot} \n ----End Example ---- \n",
        'head': 'Having a SPARQL query: {query} \n Where:\n ',
        'list': '{uri} has human-readable name "{uriLabel}."',
        'tail': '\n Transform the SPARQL query to a natural language question. Output just the transformed question'
    },
    'ru': {
        'shot': "{shot} \n ----------- \n",
        'head': 'Имея следующий SPARQL запрос: {query} \n Где:\n ',
        'list': '{uri} именуется как "{uriLabel}."',
        'tail': '\n Трансформируй SPARQL запрос в вопрос на естественном языке. Выведи только транфсормируемый вопрос'
    },
    'de': {
        'shot': "{shot} \n ----------- \n",
        'head': 'Gegeben ist die SPARQL-Anfrage: {query} \n Dabei gilt:\n ',
        'list': 'Die Bezeichnung von {uri} ist "{uriLabel}."',
        'tail': '\n Transformiere die SPARQL-Anfrage in eine Frage in natürlicher Sprache. Gib nur die transformierte Frage aus.'
    }
}

In [ ]:
prompt = make_the_prompt(db, "SELECT DISTINCT ?uri WHERE {  ?uri <http://www.wikidata.org/prop/direct/P522> <http://www.wikidata.org/entity/Q737809> . }", ZERO_SHOT_PROMPT, "en")
ask_llm(prompt=prompt, model=MODEL)

DEBUG:__main__:Found label in cache {'_id': ObjectId('655b1d660b650e9d5d021cdf'), 'uri': 'P522', 'lang': 'en', 'label': 'type of orbit'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('655b1d660b650e9d5d021ce0'), 'uri': 'Q737809', 'lang': 'en', 'label': 'heliocentric orbit'}


'Which planets are around the sun?'

# QALD-9 experiments

In [ ]:
def process_qald_dataset(dataset, MODEL=None, lang=None):
  id_list = []
  prompt_list = []
  predicted_list = []
  gs_list = []
  lang_list = []
  model_list = []
  prompt_type_list = []
  query_list = []

  for q in dataset:
    query = q["query"]["sparql"]

    prompt = make_the_prompt(db, query, PROMT_TEMPLATE, lang, dataset)
    predicted_nl = ask_llm(prompt=prompt, model=MODEL)
    gold_standard_nl_list = get_question_by_language(q, 'qald', lang)

    for gs in gold_standard_nl_list:
      id_list.append(q["id"])
      prompt_list.append(prompt)
      predicted_list.append(predicted_nl)
      gs_list.append(gs)
      lang_list.append(lang)
      model_list.append(MODEL)
      prompt_type_list.append(PROMT_TEMPLATE["type"])
      query_list.append(query)

  return pd.DataFrame.from_dict({
      "id": id_list,
      "prompt": prompt_list,
      "generated_nl": predicted_list,
      "gold_standard_nl": gs_list,
      "lang": lang_list,
      "model": model_list,
      "prompt_type": prompt_type_list,
      "query": query_list
  })

## QALD-9-plus experiments

In [ ]:
languages = ["en"]# "ru", "de"]
PROMT_TEMPLATE = ZERO_SHOT_PROMPT

for lang in languages:
  # 9-plus-test
  # QALD_9_TEST_NAME = "qald_9_plus_test_wikidata.json"
  # df = process_qald_dataset(data_dict[QALD_9_TEST_NAME]["questions"], MODEL, lang)
  # df.to_csv(QALD_9_TEST_NAME + f"_{lang}.tsv", sep='\t', index=False)

  # 9-plus-train
  QALD_9_TRAIN_NAME = "qald_9_plus_train_wikidata.json"
  df = process_qald_dataset(data_dict[QALD_9_TRAIN_NAME]["questions"], MODEL, lang)
  df.to_csv(QALD_9_TRAIN_NAME + f"_{lang}.tsv", sep='\t', index=False)

DEBUG:__main__:Found label in cache {'_id': ObjectId('655a9151e98abfa95b6b2c7b'), 'uri': 'P31', 'lang': 'en', 'label': 'instance of'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('655a9aafe98abfa95b6b2c7d'), 'uri': 'Q131436', 'lang': 'en', 'label': 'board game'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('655a9aafe98abfa95b6b2c7e'), 'uri': 'Q40984', 'lang': 'en', 'label': 'Skype'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('655a9ab0e98abfa95b6b2c7f'), 'uri': 'P178', 'lang': 'en', 'label': 'developer'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('655a9ab3e98abfa95b6b2c86'), 'uri': 'P19', 'lang': 'en', 'label': 'place of birth'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('655a9ab3e98abfa95b6b2c87'), 'uri': 'Q160544', 'lang': 'en', 'label': 'Heraklion'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('655a9ab3e98abfa95b6b2c88'), 'uri': 'Q177397', 'lang': 'en', 'label': 'area pepper'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('

## QALD-10 experiments

In [ ]:
languages = ["en", "ru", "de"]
PROMT_TEMPLATE = ONE_SHOT_PROMPT

for lang in languages:
  QALD_10_NAME = "qald_10.json"
  df = process_qald_dataset(data_dict[QALD_10_NAME]["questions"], MODEL, lang)
  df.to_csv(QALD_10_NAME + f"_{lang}.tsv", sep='\t', index=False)

DEBUG:__main__:Found label in cache {'_id': ObjectId('65620046a6a6c9552578f11f'), 'uri': 'Q761383', 'lang': 'en', 'label': 'Riemannian geometry'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('655ab473e98abfa95b6b304e'), 'uri': 'P138', 'lang': 'en', 'label': 'named after'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('65d5ecedf36c475f0d1f2e68'), 'uri': 'Q58815001', 'lang': 'en', 'label': 'Red Notice'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('655a9150e98abfa95b6b2c79'), 'uri': 'P57', 'lang': 'en', 'label': 'director'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('655a9be2e98abfa95b6b2e3b'), 'uri': 'Q44578', 'lang': 'en', 'label': 'Titanic'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('655a9150e98abfa95b6b2c79'), 'uri': 'P57', 'lang': 'en', 'label': 'director'}
DEBUG:__main__:Found label in cache {'_id': ObjectId('65d5eceff36c475f0d1f2e69'), 'uri': 'Q2745616', 'lang': 'en', 'label': 'Rawson Marshall Thurber'}
The attention mask and the pad tok